# <font color="#114b98">Catégorisez automatiquement des questions</font>

## <font color="#114b98">Notebook de test de différents modèles</font>

**Stack Overflow** est un site célèbre de questions-réponses liées au développement informatique.

L'objectif de ce projet est de développer un système de **suggestion de tags** pour ce site. Celui-ci prendra la forme d’un algorithme de machine learning qui assignera automatiquement plusieurs tags pertinents à une question.

**Livrable** : Un notebook de test de différents modèles

**Objectifs** : Comparer les modèles et générer des tags pour chacun d'entre eux

## <font color="#114b98">Sommaire</font>
[1. Chargement du jeu de données](#section_1)

[2. Approche non supervisée](#section_2)

[3. Approche supervisée](#section_3)

[4. Approche supervisée avec Word Embedding : Word2Vec](#section_4)

[5. Approche supervisée avec Word Embedding : BERT](#section_5)

[6. Approche supervisée avec Sentence Embedding : USE](#section_6)

[7. Choix du modèle pour le code final à déployer](#section_7)

## <font color="#114b98" id="section_1">1. Chargement du jeu de données</font>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import ast
import random
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
plt.rc('axes', titlesize=22) 
plt.rc('axes', labelsize=18) 
titleprops = {'fontsize':20}
textprops = {'fontsize':15}
plt.style.use('ggplot')

In [29]:
main_path = 'N:/5 - WORK/1 - Projets/Projet 5/'
files = os.listdir(main_path+'saved_ressources/')
files

['data_cleaned.csv', 'data_cleaned_wo_tokenizer.csv', 'model.h5']

In [30]:
data = pd.read_csv(main_path+'saved_ressources/'+'data_cleaned.csv', encoding='utf8')

In [31]:
data = data.applymap(lambda x: ast.literal_eval(x))

In [32]:
data.head()

,Tags,Texts
0,"[macos, emacs, scheme, interpreter, sicp]","[scheme, interpreter, interpreter, python, int..."
1,"[c#, functional-programming, delegates, lambda...","[scope, bug, compiler, voidfunction, delegate,..."
2,"[mysql, database, arrays, postgresql, stored-p...","[pass, array, procedure, pass, array, procedur..."
3,"[ios, objective-c, iphone, cocoa-touch, uibutton]","[state, buttonwithtype, figure, button, state,..."
4,"[database, nhibernate, exception-handling, con...","[check, application, column, constraint, colum..."


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159599 entries, 0 to 159598
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Tags    159599 non-null  object
 1   Texts   159599 non-null  object
dtypes: object(2)
memory usage: 2.4+ MB


Le jeu de données est trop important pour les temps de calculs à ma disposition.

J'ai deux possibilités : 
- prendre un sample d'observations aléatoirement
- prendre les observations pour lesquelles la similarité entre les deux colonnes est importante

In [34]:
# data_sample = data.sample(20000)
# data_sample.reset_index(inplace=True, drop=True)
# print(data_sample.shape)
# data_sample.head()

Je choisi la seconde option afin de pouvoir regarder la pertinence des tags que mes modèles vont proposer.

In [35]:
from typing import List

In [36]:
def jaccard_similarity(list1: List[str], list2: List[str]) -> float:
    set1 = set(list1)
    set2 = set(list2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity

def get_highest_similarity_rows(data: pd.DataFrame, col1, col2, n):
    data["jaccard_similarity"] = data.apply(lambda x: jaccard_similarity(x[col1], x[col2]), axis=1)
    data = data.sort_values(by="jaccard_similarity", ascending=False)
    return data.head(n)


data_sample = get_highest_similarity_rows(data, "Tags", "Texts", 20000)
data_sample.drop(['jaccard_similarity'], axis=1, inplace=True)
data_sample.reset_index(inplace=True, drop=True)
print(data_sample.shape)
data_sample.head()

(20000, 2)


,Tags,Texts
0,"[javascript, jquery, string, date, object]","[jquery, javascript, convert, date, string, da..."
1,"[listview, javafx, tableview, scrollbar, hide]","[hide, scrollbar, listview, javafx, hide, scro..."
2,"[jpa, merge, entitymanager, persist, java-pers...","[jpa, entitymanager, merge, entitymanager, mer..."
3,"[python, django, image, gallery, portfolio]","[create, image, gallery, create, portfolio, dj..."
4,"[python, authentication, proxy, https, ntlm]","[python, https, proxy, ntlm, authentication, s..."


In [37]:
from sklearn.model_selection import train_test_split

Je séprare le dataset obtenu en deux parties, afin de réaliser des essais surle X_train et de tester mes modèles sur le X_test.

In [38]:
X = data_sample[['Texts']]
y = data_sample[['Tags']]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True, random_state=42)

In [40]:
train_indexes = X_train.index
test_indexes = X_test.index

In [41]:
train_df = pd.DataFrame({'Texts': X_train['Texts'].tolist(), 'Tags': y_train['Tags'].tolist()})

In [42]:
train_df

,Texts,Tags
0,"[web, worker, way, worker, thread, module, sup...","[java, html, gwt, web, web-worker]"
1,"[rvm, rbenv, work, rvm, admit, knowledge]","[ruby-on-rails, ruby, rubygems, rvm, rbenv]"
2,"[javascript, javascript, application, way, jav...","[javascript, tcp, udp, websocket, packet]"
3,"[crash, class, app, iphone, simulator, simulat...","[iphone, ios5, ios6, autolayout, nslayoutconst..."
4,"[import, database, phpmyadmin, file, size, imp...","[mysql, database, apache, phpmyadmin, wampserver]"
...,...,...
9995,"[image, note, question, way, question, regex, ...","[php, .net, regex, perl, pcre]"
9996,"[advice, automate, rest, kind, rest, dept, aut...","[python, api, rest, testing, automated-tests]"
9997,"[custom, camera, view, use, camera, iphone, vi...","[ios, iphone, xcode, swift, camera]"
9998,"[expression, expression, lambda, translate, st...","[string, macros, lambda, scheme, expression]"


In [43]:
texts_list = train_df["Texts"].to_list()
tags_list = train_df["Tags"].to_list()
flat_texts = [" ".join(text) for text in texts_list]
flat_tags = [" ".join(tag) for tag in tags_list]
vocabulary_texts = list(set([word for item in texts_list for word in item]))
vocabulary_tags = list(set([word for item in tags_list for word in item]))

In [44]:
test_df = pd.DataFrame({'Texts': X_test['Texts'].tolist(), 'Tags': y_test['Tags'].tolist()})

In [45]:
test_texts_list = test_df["Texts"].to_list()
test_tags_list = test_df["Tags"].to_list()
test_flat_texts = [" ".join(text) for text in test_texts_list]
test_flat_tags = [" ".join(tag) for tag in test_tags_list]
test_vocabulary_texts = list(set([word for item in texts_list for word in item]))
test_vocabulary_tags = list(set([word for item in tags_list for word in item]))

## <font color="#114b98" id="section_2">2. Approche non supervisée</font>

### Dataset d'essais

In [46]:
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, accuracy_score, jaccard_score, precision_score
from sklearn.preprocessing import MultiLabelBinarizer
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Nmf
from gensim.models.ldamodel import LdaModel
from gensim.matutils import corpus2dense
from collections import Counter

LDA (Latent Dirichlet Allocation) est une technique de topic modeling qui permet de découvrir les thèmes cachés (ou "latents") dans un ensemble de textes. Elle permet de regrouper des textes qui traitent des mêmes sujets.

La classe LdaModel de gensim est basée sur l'algorithme d'allocation latente de Dirichlet (LDA), qui est un modèle probabiliste génératif utilisé pour découvrir les sujets cachés dans un corpus de textes. La classe LatentDirichletAllocation de scikit-learn est également basée sur l'algorithme LDA, mais elle peut avoir des différences en termes d'implémentation, comme l'algorithme d'optimisation utilisé ou les paramètres disponibles.

NMF (Non-negative Matrix Factorization) est une autre technique de topic modeling qui permet de décomposer une matrice document-terme en deux matrices de facteurs non-négatifs. Elle est souvent utilisée pour découvrir les thèmes cachés dans des textes.

La classe gensim Nmf est basée sur l'algorithme de factorisation de matrice non-négative, qui est différente de la classe NMF de scikit-learn, qui est basée sur la méthode de gradient projeté.

In [47]:
def determine_optimal_num_topics(data, vectorizer, n_topics_range, texts_list):
    """
    Given data, a vectorizer, a range of number of topics to test, and the list of texts,
    applies the models to the data and plots the silhouette and coherence scores to help 
    determine the optimal number of topics.
    
    """

    # Vectorize the data
    data = vectorizer.fit_transform(data)
    dictionary = Dictionary(texts_list)
    corpus = [dictionary.doc2bow(txt) for txt in texts_list]

    # Initialize lists to store scores for LDA and NMF
    lda_scores = []
    nmf_scores = []
    coherence_nmf = []
    coherence_lda = []

    # Loop through the range of number of topics
    for n_topics in n_topics_range:
        
        # Calculate the silhouette score for the LDA model
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=1000)
        lda.fit(data)
        topic_assignments = lda.transform(data)
        labels = np.argmax(topic_assignments, axis=1)
        lda_scores.append(silhouette_score(topic_assignments, labels, metric='euclidean'))
        
        # Calculate the silhouette score for the NMF model
        nmf = NMF(n_components=n_topics, max_iter=1000)
        nmf.fit(data)
        topic_assignments = nmf.transform(data)
        labels = np.argmax(topic_assignments, axis=1)
        nmf_scores.append(silhouette_score(topic_assignments, labels, metric='euclidean'))
        
        # Calculate the coherence score for the LDA model
        lda = LdaModel(corpus, num_topics=n_topics, id2word=dictionary)
        cm_lda = CoherenceModel(model=lda, texts=texts_list, dictionary=dictionary, coherence='c_v')
        coherence_lda.append(cm_lda.get_coherence())
            
        # Calculate the coherence score for the NMF model
        nmf = Nmf(corpus, num_topics=n_topics, id2word=dictionary)
        cm_nmf = CoherenceModel(model=nmf, texts=texts_list, dictionary=dictionary, coherence='c_v')
        coherence_nmf.append(cm_nmf.get_coherence())
     
    scores = pd.DataFrame(columns=['topics_silhouette',
                                   'score_silhouette',
                                   'topics_coherence',
                                   'score_coherence'], 
                          index=['LDA', 'NMF'])

    scores['topics_silhouette'] = [n_topics_range[np.argmax(lda_scores)], n_topics_range[np.argmax(nmf_scores)]]
    scores['score_silhouette'] = [max(lda_scores), max(nmf_scores)]
    scores['topics_coherence'] = [n_topics_range[np.argmax(coherence_lda)], n_topics_range[np.argmax(coherence_nmf)]]
    scores['score_coherence'] = [max(coherence_lda), max(coherence_nmf)]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    plt.suptitle('Scores de Silhouette et de Coherence pour LDA et NMF avec {}'.format(str(vectorizer).split('(')[0]))
    
    ax1.plot(n_topics_range, lda_scores, label='LDA')
    ax1.plot(n_topics_range, nmf_scores, label='NMF')
    ax1.set_xlabel('Number of Topics')
    ax1.set_ylabel('Silhouette score')
    ax1.legend()

    ax2.plot(n_topics_range, coherence_lda, label='LDA')
    ax2.plot(n_topics_range, coherence_nmf, label='NMF')
    ax2.set_xlabel('Number of Topics')
    ax2.set_ylabel('Coherence score')
    ax2.legend()

    plt.show()
    
    return scores

In [48]:
# Define the range of number of topics to test
n_topics_range = range(2, 5, 2)

CountVectorizer() est une implémentation de l'approche bag-of-words pour la vectorisation de textes. Il convertit un ensemble de documents en un tableau de compte de mots (ou un sac de mots), où chaque ligne représente un document et chaque colonne représente un mot. Le nombre dans chaque cellule est le nombre de fois où le mot correspondant est présent dans le document correspondant.

In [49]:
# Initialize the vectorizer
vectorizer = CountVectorizer()

In [ ]:
# nmf_and_lda_models_with_CountVectorizer = determine_optimal_num_topics(flat_texts,
#                                                                        vectorizer,
#                                                                        n_topics_range,
#                                                                        texts_list)

In [ ]:
nmf_and_lda_models_with_CountVectorizer

TF-IDF (term frequency-inverse document frequency) est une technique utilisée pour pondérer les termes dans les textes en fonction de leur fréquence d'apparition. Elle permet de donner plus de poids aux termes qui apparaissent fréquemment dans un document mais rarement dans l'ensemble des documents.

In [ ]:
# Initialize the vectorizer
vectorizer = TfidfVectorizer(vocabulary=vocabulary_texts)

In [ ]:
# nmf_and_lda_models_with_TfidfVectorizer= determine_optimal_num_topics(flat_texts,
#                                                                       vectorizer,
#                                                                       n_topics_range,
#                                                                       texts_list)

In [ ]:
nmf_and_lda_models_with_TfidfVectorizer

Le score de silhouette mesure la similarité d'un objet à son propre groupe par rapport aux autres groupes et généralement, plus il est proche de 1, meilleure est la classification. Le score de cohérence mesure à quel point les sujets sont "interprétables par les humains", généralement plus proche de 1, meilleur c'est.

Dans notre situation, lorsque le nombre de sujets augmente, ils davantage "interprétables par les humains".

Nous devons maintenant essayer d'obtenir des tags en utilisant ces méthodes.

Je choisis d'utiliser uniquement LDA pour la suite car c'est la méthode qui obtient les meilleurs scores de silhouette.

Le paramètre min_df définit le nombre minimum de documents dans lesquels un mot doit être présent pour être inclus dans le vocabulaire.

In [50]:
min_df=25

Le paramètre max_df définit la fréquence maximale d'un mot en pourcentage de tous les documents. 

In [51]:
max_df=0.2

Je choisis le nombre de topics au regard des résultats précédents.

In [52]:
n_topics = 11

In [53]:
def get_tags_from_text(texts_list, flat_texts, n_topics, vocabulary_texts, min_df, max_df):
    pred_tags_gensim = list()
    pred_tags_sklearn = list()
    pred_tags_tfidf = list()
    pred_tags_count = list()
    
    # Predict tags using LdaModel (gensim) without bow or TF-IDF 
    dictionary = Dictionary(texts_list)
    corpus = [dictionary.doc2bow(txt) for txt in texts_list]
    lda = LdaModel(corpus, num_topics=n_topics, id2word=dictionary, random_state=42)
    for text in texts_list:
        bow = dictionary.doc2bow(text)
        topics = lda.get_document_topics(bow, minimum_probability=0)
        topic_id, prob = max(topics, key=lambda x: x[1])
        topic_words = [w for w, p in lda.show_topic(topic_id, topn=5)]
        pred_tags_gensim.append(topic_words)

    # Predict tags using LDA (sklearn) without bow or TF-IDF    
    corpus_dense = corpus2dense(corpus, num_terms=len(dictionary)).T
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(corpus_dense)
    for text in texts_list:
        bow = dictionary.doc2bow(text)
        dense_bow = corpus2dense([bow], num_terms=len(dictionary)).T[0]
        dense_bow = np.reshape(dense_bow, (1, -1))
        topic_distribution = lda.transform(dense_bow)
        topic_id = topic_distribution.argmax()
        top_words_indices = np.argsort(-lda.components_[topic_id])[:5]
        topic_words = [dictionary[i] for i in top_words_indices]
        pred_tags_sklearn.append(topic_words)     
               
    # Predict tags using LdaModel with TF-IDF 
    vectorizer = TfidfVectorizer(vocabulary=vocabulary_texts, min_df=min_df, max_df=max_df)
    bow = vectorizer.fit_transform(flat_texts)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    topics = lda.fit_transform(bow)
    for i in range(len(texts_list)):
        topic_id = topics.argmax(axis=1)[i]
        dense_bow_matrix = bow.toarray()
        top_words_indices = dense_bow_matrix[i].argsort()[-5:][::-1]
        topic_words = [list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(i)] for i in top_words_indices]
        pred_tags_tfidf.append(topic_words)
        
    # Predict tags using LdaModel with CountVectorizer 
    vectorizer = CountVectorizer(vocabulary=vocabulary_texts, min_df=min_df, max_df=max_df)
    bow = vectorizer.fit_transform(flat_texts)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    topics = lda.fit_transform(bow)
    for i in range(len(texts_list)):
        topic_id = topics.argmax(axis=1)[i]
        dense_bow_matrix = bow.toarray()
        top_words_indices = dense_bow_matrix[i].argsort()[-5:][::-1]
        topic_words = [list(vectorizer.vocabulary_.keys())[list(vectorizer.vocabulary_.values()).index(i)] for i in top_words_indices]
        pred_tags_count.append(topic_words)           
        
    return pred_tags_gensim, pred_tags_sklearn, pred_tags_tfidf, pred_tags_count

In [ ]:
pred_tags_gensim, pred_tags_sklearn, pred_tags_tfidf, pred_tags_count = get_tags_from_text(texts_list,
                                                                                           flat_texts,
                                                                                           n_topics,
                                                                                           vocabulary_texts,
                                                                                           min_df,
                                                                                           max_df)

In [ ]:
tags_list[0:5]

In [ ]:
pred_tags_gensim[0:5]

In [ ]:
pred_tags_sklearn[0:5]

In [ ]:
pred_tags_tfidf[0:5]

In [ ]:
pred_tags_count[0:5]

Il semble que CountVectorizer et TfidfVectorizer prédisent des tags assez similaires.

In [ ]:
true_tags = tags_list
pred_tags_list = [pred_tags_gensim, pred_tags_sklearn, pred_tags_tfidf, pred_tags_count]
pred_names = ["gensim", "sklearn", "tfidf", "count"]

In [ ]:
mlb = MultiLabelBinarizer()
true_tags_bin = mlb.fit_transform(true_tags)
pred_tags_bin_list = [mlb.transform(pred_tags) for pred_tags in pred_tags_list]

In [ ]:
def evaluate_predictions(true_tags, pred_tags_bin_list, pred_names):
    f1_scores = []
    jaccard_scores = []
    accuracy_scores = []
    precision_scores = []
    for pred_tags in pred_tags_bin_list:
        f1_scores.append(f1_score(true_tags, pred_tags, average='samples'))
        jaccard_scores.append(jaccard_score(true_tags, pred_tags, average='samples'))
        accuracy_scores.append(accuracy_score(true_tags, pred_tags))
        precision_scores.append(precision_score(true_tags, pred_tags, average='samples'))
        
    metrics = {"F1": f1_scores, "Jaccard": jaccard_scores, "Accuracy": accuracy_scores, "Precision": precision_scores}
    metrics_df = pd.DataFrame(metrics, index=pred_names)
    
    fig, axes = plt.subplots(2, 2, figsize=(15,10))
    axes = axes.ravel()
    for i, metric in enumerate(metrics.keys()):
        sns.barplot(data=metrics_df, x=metrics_df.index, y=metric, ax=axes[i])
        axes[i].set(ylabel=metric)
    plt.show()
    return metrics_df

In [ ]:
evaluate_predictions(true_tags_bin, pred_tags_bin_list, pred_names)

In [ ]:
def evaluate_predictions_one_to_one(true_tags, pred_tags_list, pred_names):
    f1_scores = []
    jaccard_scores = []
    accuracy_scores = []
    precision_scores = []
    for pred_tags in pred_tags_list:
        f1_scores.append(f1_score(true_tags, pred_tags, average='samples'))
        jaccard_scores.append(jaccard_score(true_tags, pred_tags, average='samples'))
        accuracy_scores.append(sum([len(set(true_tags[i]).intersection(pred_tags[i]))/len(true_tags[i]) for i in range(len(true_tags))])/len(true_tags))
        precision_scores.append(precision_score(true_tags, pred_tags, average='samples'))
        
    metrics = {"F1": f1_scores, "Jaccard": jaccard_scores, "Accuracy": accuracy_scores, "Precision": precision_scores}
    metrics_df = pd.DataFrame(metrics, index=pred_names)
    
    fig, axes = plt.subplots(2, 2, figsize=(15,10))
    axes = axes.ravel()
    for i, metric in enumerate(metrics.keys()):
        sns.barplot(data=metrics_df, x=metrics_df.index, y=metric, ax=axes[i])
        axes[i].set(ylabel=metric)
    plt.show()
    return metrics_df

In [ ]:
evaluate_predictions_one_to_one(true_tags_bin, pred_tags_bin_list, pred_names)

In [ ]:
def plot_similar_tags(true_tags, pred_tags, method):
    similar_counts = []
    for pred_tags, true_tags in zip(pred_tags, true_tags):
        similar_words = set(pred_tags) & set(true_tags)
        similar_counts.append(len(similar_words))

    counter = Counter(similar_counts)
    counter = dict(sorted(counter.items()))
    
    # Add missing keys to counter with value 0
    keys = set(range(min(counter.keys()), max(counter.keys()) + 1))
    missing_keys = keys - set(counter.keys())
    for key in missing_keys:
        counter[key] = 0
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    fig.suptitle(f"Similarité des tags avec la méthode {method}", fontsize=18, fontweight='bold', y=1.05)
    axs[0].bar(sorted(counter.keys()), counter.values())
    axs[0].set_xticks(np.arange(min(counter.keys()), max(counter.keys())+1))
    axs[0].set_xticklabels(counter.keys(), rotation=0)
    axs[0].set_xlabel('Nombre de tags similaires', fontsize=14)
    axs[0].set_ylabel("Nombre d'observations", fontsize=14)
    axs[0].set_title("Nombre d'observations avec un\n nombre de tags similaires", fontsize=16)   
    axs[1].pie(counter.values(), labels=counter.keys(), autopct='%1.1f%%', pctdistance=0.8)
    axs[1].legend(title='Tags\nSimilaires', bbox_to_anchor=(1, 0.9), prop={'size': 8},title_fontsize=10)
    axs[1].set_title("Pourcentage d'observations avec \n un nombre de tags similaires", fontsize=16)

    plt.show()

### Dataset de tests

In [ ]:
pred_gensim, pred_sklearn, pred_tfidf, pred_count = get_tags_from_text(test_texts_list,
                                                                       test_flat_texts,
                                                                       n_topics,
                                                                       test_vocabulary_texts,
                                                                       min_df,
                                                                       max_df)

In [ ]:
plot_similar_tags(test_tags_list, pred_gensim, 'LDA Gensim')

In [ ]:
plot_similar_tags(test_tags_list, pred_sklearn, 'LDA Sklearn')

In [ ]:
plot_similar_tags(test_tags_list, pred_tfidf, 'LDA + TFIDF')

In [ ]:
plot_similar_tags(test_tags_list, pred_count, 'LDA + Count')

In [ ]:
# AJOUT PCA !

## <font color="#114b98" id="section_3">3. Approche supervisée</font>

### Dataset d'essais

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mlb = MultiLabelBinarizer(classes=vocabulary_tags)

In [ ]:
tags_mlb = mlb.fit_transform(flat_tags)

In [ ]:
classifiers = [LogisticRegression(random_state=42, max_iter=300, tol=1e-5),
               SGDClassifier(random_state=42, max_iter=300, tol=1e-5),
               RandomForestClassifier(random_state=42),
               KNeighborsClassifier(),
               MultinomialNB()]

 - Accuracy: mesure de combien de prédictions faites par le modèle sont correctes
 - Precision: mesure combien des prédictions positives faites par le modèle sont effectivement correctes. Un score de précision élevé signifie que le modèle fait peu de prédictions positives fausses.
 - Recall: mesure combien des exemples positifs réels sont correctement prédits par le modèle. Un score de rappel élevé signifie que le modèle est capable de trouver la plupart des exemples positifs.
 - F1 Score: mesure de l'exactitude d'un modèle, il est un moyen harmonique de précision et de rappel. Il varie de 0 à 1, où un score proche de 1 indique une meilleure performance et un score proche de 0 indique une performance moins bonne. 
 - Jaccard Score: mesure de la similarité entre les deux ensembles de prédictions et de vraies étiquettes. Il varie de 0 à 1, où un score proche de 1 indique une très grande similitude et un score proche de 0 indique une grande dissimilarité.

In [ ]:
def calculate_supervised_scores(flat_texts, tags_mlb, vectorizer, classifiers):
    
    # Create an empty dataframe to store the results
    results_df = pd.DataFrame(columns=['Classifier', 'Accuracy', 'Precision', 'Recall',
                                       'F1 Score', 'Jaccard Score', 'Time (s)'])

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(flat_texts, tags_mlb, test_size=0.2, random_state=42)
    
    # Vectorize X_train and X_test
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    # Fit an independent model for each class using the OneVsRestClassifier wrapper.
    for classifier in classifiers:
        start_time = time.time()
        ovrc = OneVsRestClassifier(classifier)
        ovrc.fit(X_train, y_train)
        y_pred_ovrc = ovrc.predict(X_test)
        end_time = time.time()
        
        accuracy = round(accuracy_score(y_test, y_pred_ovrc), 4)
        precision = round(precision_score(y_test, y_pred_ovrc, average='samples'), 4)
        recall = round(recall_score(y_test, y_pred_ovrc, average='samples'), 4)
        f1 = round(f1_score(y_test, y_pred_ovrc, average='samples'), 4)
        jaccard = round(jaccard_score(y_test, y_pred_ovrc, average='samples'), 4)
        time_taken = round(end_time - start_time, 4)
        
        results_df = results_df.append({'Classifier': str(classifier).split('(')[0], 
                                       'Accuracy': accuracy, 
                                       'Precision': precision, 
                                       'Recall': recall, 
                                       'F1 Score': f1, 
                                       'Jaccard Score': jaccard,
                                       'Time (s)': time_taken}, 
                                       ignore_index=True)
                                       
        print('Results for classifier:', classifier)
        print("Accuracy : ", accuracy)
        print("Precision : ", precision)
        print("Recall : ", recall)
        print("F1 Score : ", f1)
        print("Jaccard Score:", jaccard)
        print("Time (s):", time_taken)
        print('\n')
        
    return results_df

In [ ]:
# Initialize the vectorizer
vectorizer = CountVectorizer()

In [ ]:
results_df_CountVectorizer = calculate_supervised_scores(flat_texts,
                                                         tags_mlb,
                                                         vectorizer,
                                                         classifiers)

In [ ]:
results_df_CountVectorizer

Selon ces résultats, il semble que le RandomForestClassifier a les meilleures performances globales, avec la plus grande précision, rappel et score F1. Le classificateur de régression logistique se comporte également bien, avec une précision légèrement inférieure.

Cependant, le RandomForestClassifier est aussi le seul à nécessiter un temps d'entraînement aussi long.

In [ ]:
def plot_results(results_df):
    
    # Create a figure with 5 subplots
    fig, axs = plt.subplots(2, 3, figsize=(20,10))
    # Set a color palette
    my_palette = sns.color_palette("husl", 5)
    
    # Set the x-axis to be a range of numerical values
    x = range(len(results_df))
    scoring_methods = ['Accuracy','Precision','Recall','F1 Score','Jaccard Score', 'Time (s)']
    
    # Create a subplot for each scoring method
    for i, scoring_method in enumerate(scoring_methods):
        sns.barplot(x='Classifier', 
                    y=scoring_method, 
                    data=results_df, 
                    ax=axs[i // 3, i % 3], 
                    palette=my_palette, 
                    label=scoring_method)
    
    # Add classifier names to x-axis
    for i in range(2):
        for j in range(3):
            axs[i,j].set_title(scoring_methods[i*3+j])
            axs[i,j].set_xticks(x)
            axs[i,j].set_xlabel('')
            
            if j == 0:
                axs[i,j].set_ylabel('Score')
            else:
                axs[i,j].set_ylabel('')
            
            if i == 1:
                axs[i,j].set_xticklabels(results_df['Classifier'], rotation=90)
            else:
                axs[i,j].set_xticklabels('')
            
    plt.show()

In [ ]:
# Call the plot_results function
plot_results(results_df_CountVectorizer)

In [ ]:
# Initialize the vectorizer
vectorizer = TfidfVectorizer(vocabulary=vocabulary_texts)

In [ ]:
results_df_TfidfVectorizer = calculate_supervised_scores(flat_texts,
                                                         tags_mlb,
                                                         vectorizer,
                                                         classifiers)

In [ ]:
results_df_TfidfVectorizer

In [ ]:
# Call the plot_results function
plot_results(results_df_TfidfVectorizer)

Le RandomForestClassifier obtient les meilleurs scores, peu importe le vectorizer utilisé.

In [ ]:
results_RFC = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Jaccard Score'])
results_RFC = results_RFC.append(results_df_CountVectorizer.iloc[2,1:]).reset_index(drop=True)
results_RFC = results_RFC.append(results_df_TfidfVectorizer.iloc[2,1:]).reset_index(drop=True)
results_RFC['Classifier'] = ['CountVectorizer', 'TfidfVectorizer']
results_RFC

In [ ]:
# Call the plot_results function
plot_results(results_RFC)

CountVectorizer permet d'avoir des performances similaires avec un temps d'entraînement plus court que TfidfVectorizer.

### Dataset de tests

## <font color="#114b98" id="section_4">4. Approche supervisée avec Word Embedding : Word2Vec</font>

### Dataset d'essais

In [ ]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(texts_list,
                                                    tags_mlb,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
# Train the Word2Vec model on your text data
w2v_model = Word2Vec(X_train, vector_size=1000, window=5, min_count=5, workers=4)

In [ ]:
# Create a vocabulary of only the words in the text data that are in the word2vec model
vocab = set(w2v_model.wv.key_to_index.keys())

In [ ]:
# Filter the text data to only include words in the vocabulary
X_train = [[word for word in sublist if word in vocab] for sublist in X_train]
X_test = [[word for word in sublist if word in vocab] for sublist in X_test]

In [ ]:
# Remove any observations that have no words in the vocabulary
train_removed_indexes = []
test_removed_indexes = []
for i, sublist in enumerate(X_train):
    if not any(word in vocab for word in sublist):
        train_removed_indexes.append(i)
for i, sublist in enumerate(X_test):
    if not any(word in vocab for word in sublist):
        test_removed_indexes.append(i)

In [ ]:
X_train = [x for i, x in enumerate(X_train) if i not in train_removed_indexes]
X_test = [x for i, x in enumerate(X_test) if i not in test_removed_indexes]
y_train = [x for i, x in enumerate(y_train) if i not in train_removed_indexes]
y_test = [x for i, x in enumerate(y_test) if i not in test_removed_indexes]

In [ ]:
# Create embeddings for train and test data
X_train_embedded = [np.mean([w2v_model.wv[word] for word in sentence], axis=0) for sentence in X_train]
X_test_embedded = [np.mean([w2v_model.wv[word] for word in sentence], axis=0) for sentence in X_test]

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(random_state=42))

In [ ]:
# Fit the classifier on the train data
clf.fit(X_train_embedded, y_train)

In [ ]:
# Make predictions on the test data
y_pred = clf.predict(X_test_embedded)

In [ ]:
# Calculate scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')
jaccard = jaccard_score(y_test, y_pred, average='samples')

# Print the results
print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-Score: {:.2f}".format(f1))
print("Jaccard Score: {:.2f}".format(jaccard))

In [ ]:
classifiers = [LogisticRegression(random_state=42, max_iter=300, tol=1e-5),
               SGDClassifier(random_state=42, max_iter=300, tol=1e-5),
               RandomForestClassifier(random_state=42),
               KNeighborsClassifier()]

In [ ]:
def calculate_supervised_word2vec(X_train, X_test, y_train, y_test, classifiers):
    
    # Create an empty dataframe to store the results
    results_df = pd.DataFrame(columns=['Classifier', 'Accuracy', 'Precision', 'Recall',
                                       'F1 Score', 'Jaccard Score', 'Time (s)'])
    
    # Fit an independent model for each class using the OneVsRestClassifier wrapper.
    for clf in classifiers:
        start_time = time.time()
        ovrc = OneVsRestClassifier(clf)
        ovrc.fit(X_train, y_train)
        y_pred_ovrc = ovrc.predict(X_test)
        end_time = time.time()
        
        accuracy = round(accuracy_score(y_test, y_pred_ovrc), 4)
        precision = round(precision_score(y_test, y_pred_ovrc, average='samples'), 4)
        recall = round(recall_score(y_test, y_pred_ovrc, average='samples'), 4)
        f1 = round(f1_score(y_test, y_pred_ovrc, average='samples'), 4)
        jaccard = round(jaccard_score(y_test, y_pred_ovrc, average='samples'), 4)
        time_taken = round(end_time - start_time, 4)
        
        results_df = results_df.append({'Classifier': str(clf).split('(')[0], 
                                       'Accuracy': accuracy, 
                                       'Precision': precision, 
                                       'Recall': recall, 
                                       'F1 Score': f1, 
                                       'Jaccard Score': jaccard,
                                       'Time (s)': time_taken}, 
                                       ignore_index=True)
        
    return results_df

In [ ]:
results_df_word2vec = calculate_supervised_word2vec(X_train_embedded,
                                                    X_test_embedded,
                                                    y_train,
                                                    y_test,
                                                    classifiers)

In [ ]:
# Call the plot_results function
plot_results(results_df_word2vec)

### Dataset de tests

## <font color="#114b98" id="section_5">5. Approche supervisée avec Word Embedding : BERT</font>

### Dataset d'essais

In [ ]:
import torch
import tensorflow_hub as hub
import tensorflow as tf
import transformers
import tokenization
from transformers import BertTokenizer, AutoModel, BertTokenizerFast
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch import nn
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from tqdm import tqdm

In [ ]:
vocabulary_tags = set([word for word, count in Counter([word for item in tags_list for word in item]).most_common() if count >= 20])

In [ ]:
# pip install  torch===1.7.1+cu110 torchvision===0.8.2+cu110 torchaudio===0.7.2 --user -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
texts = data_sample['Texts'].copy()
tags = data_sample['Tags'].copy()

In [ ]:
for i in range(len(texts)):
    texts[i] = " ".join(texts[i])
    tags[i] = " ".join(tags[i])

In [ ]:
mlb = MultiLabelBinarizer()
tags_bin = mlb.fit_transform(tags_list)

In [ ]:
train_text, test_text, train_labels, test_labels = train_test_split(texts, 
                                                                    tags_bin, 
                                                                    test_size=0.2, 
                                                                    random_state=42)

In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab_file, do_lower_case)

In [ ]:
tokens_train = bert_encode(train_text.values, tokenizer, max_len=100)
tokens_test = bert_encode(test_text.values, tokenizer, max_len=100)

In [ ]:
tokenizer

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    out = Dense(len(train_labels[0]), activation='sigmoid')(clf_output)    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

In [ ]:
model = build_model(bert_layer, max_len=100)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# use one-hot encoded labels for training
train_history = model.fit(
    tokens_train, train_labels,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
)

In [ ]:
sns.set_style("darkgrid")

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(x=np.arange(1,11), y=train_history.history['loss'], label="Training Loss", ax=axs[0])
sns.lineplot(x=np.arange(1,11), y=train_history.history['val_loss'], label="Validation Loss", ax=axs[0])
axs[0].set_title("Loss")
axs[0].set_xticks(np.arange(1,11))
axs[0].set_xlabel("Epoch")

sns.lineplot(x=np.arange(1,11), y=train_history.history['binary_accuracy'], label="Training Accuracy", ax=axs[1])
sns.lineplot(x=np.arange(1,11), y=train_history.history['val_binary_accuracy'], label="Validation Accuracy", ax=axs[1])
axs[1].set_title("Accuracy")
axs[1].set_xticks(np.arange(1,11))
axs[1].set_xlabel("Epoch")

plt.tight_layout()
plt.show()

In [ ]:
model.evaluate(tokens_test, test_labels)

In [ ]:
predictions = []

for text in tqdm(flat_texts):
    preds = model.predict(bert_encode(text, tokenizer, max_len=100))
    indices = np.argsort(preds)[0][-5:]
    preds[0, indices] = 1
    preds[np.where(preds != 1)] = 0
    decoded = mlb.inverse_transform(preds)
    predictions.append(decoded[0])

In [ ]:
predictions[0:5]

In [ ]:
def convert_tags(row):
    tags = row.split()
    tags = [f"'{tag}'" for tag in tags]
    return tuple(tags)

converted_tags = tags.apply(convert_tags)

In [ ]:
converted_tags

In [ ]:
# parsed_true_tags = [ast.literal_eval(tag[1:-1]) for tag in tags.to_list()]

In [ ]:
# parsed_true_tags[0:5]

In [ ]:
plot_similar_tags(converted_tags, predictions, 'BERT')

### Dataset de tests

## <font color="#114b98" id="section_6">6. Approche supervisée avec Sentence Embedding : USE</font>

### Dataset d'essais

In [ ]:
from rake_nltk import Rake
# from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [ ]:
data_use = pd.read_csv(main_path+'saved_ressources/'+'data_cleaned_wo_tokenizer.csv', encoding='utf8')

In [ ]:
sentences = data_use['Texts'].to_list()
tags = data_use['Tags'].to_list()

In [ ]:
extracted_sentences = [sentences[i] for i in train_indexes]
extracted_tags = [tags[i] for i in train_indexes]
parsed_true_tags = [ast.literal_eval(tags[1:-1]) for tags in extracted_tags]

In [ ]:
def extract_keywords(input_text):
    # Load the USE model
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    all_keywords = []
    # Encode all the sentences in the input text
    embeddings = model(input_text)
    # Compute the cosine similarity between all the sentences
    similarity_matrix = cosine_similarity(embeddings)
    # Iterate over the list of sentences
    for i in range(len(input_text)):
        # Find the most similar sentences
        most_similar = np.argsort(-similarity_matrix[i])[1:6]
        # Combine the most similar sentences with the current sentence
        text = ' '.join([input_text[j] for j in most_similar])
        # Extract the keywords from the combined text using RAKE
        keyword_extractor = Rake()
        keyword_extractor.extract_keywords_from_text(text)
        word_degrees = keyword_extractor.get_word_degrees()
        sorted_word_degrees = sorted(word_degrees.items(), key=lambda x: x[1], reverse=True)
        keywords = [word for word, degree in sorted_word_degrees[:5]]
        all_keywords.append(keywords)
    return all_keywords

In [ ]:
keywords = extract_keywords(extracted_sentences)

In [ ]:
keywords[0:5]

In [ ]:
parsed_true_converted_tags = [list(tag) for tag in parsed_true_tags]

In [ ]:
parsed_true_converted_tags[0:5]

In [ ]:
plot_similar_tags(parsed_true_converted_tags, keywords, 'USE')

### Dataset de tests

In [ ]:
test_extracted_sentences = [sentences[i] for i in test_indexes]
test_extracted_tags = [tags[i] for i in test_indexes]
test_parsed_true_tags = [ast.literal_eval(tags[1:-1]) for tags in test_extracted_tags]
test_keywords = extract_keywords(test_extracted_sentences)
test_parsed_true_converted_tags = [list(tag) for tag in test_parsed_true_tags]

In [ ]:
test_keywords[0:5]

In [ ]:
test_parsed_true_converted_tags[0:5]

In [ ]:
plot_similar_tags(test_parsed_true_converted_tags, test_keywords, 'USE')

## <font color="#114b98" id="section_7">7. Choix du modèle pour le code final à déployer</font>

Nous allons maintenant comparer les différents modèles à l'aide de leurs résultats sur le dataset de tests.